# Quantum ESPRESSO Workshop - Part 2: SCF Calculation Fundamentals

## Learning Objectives
1. Write a proper SCF input file from scratch
2. Understand each input parameter's physical meaning
3. Run an SCF calculation and monitor its progress
4. Parse and interpret the output file completely
5. Extract key physical quantities

---

## 1. Setup and Imports

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import os

# Set up paths
WORKSHOP_ROOT = Path('/home/claude/qe_workshop')
PSEUDO_DIR = WORKSHOP_ROOT / 'pseudopotentials'
OUTPUT_DIR = WORKSHOP_ROOT / 'outputs'

# Create working directory for this notebook
WORK_DIR = OUTPUT_DIR / '02_scf_basics'
WORK_DIR.mkdir(exist_ok=True)
(WORK_DIR / 'tmp').mkdir(exist_ok=True)

print(f"Working directory: {WORK_DIR}")
print(f"Pseudopotential directory: {PSEUDO_DIR}")

---

## 2. Building the SCF Input File Step by Step

We'll build the input file piece by piece, explaining each section thoroughly.

### 2.1 The &CONTROL Namelist

This namelist controls the **job type** and **I/O settings**.

In [ ]:
# CONTROL namelist parameters explained
control_params = {
    'calculation': {
        'value': 'scf',
        'description': 'Type of calculation',
        'options': ['scf', 'relax', 'vc-relax', 'bands', 'nscf', 'md']
    },
    'prefix': {
        'value': 'silicon',
        'description': 'Prefix for all output files (used by post-processing)'
    },
    'outdir': {
        'value': './tmp',
        'description': 'Directory for large temporary files (wavefunctions, charge)'
    },
    'pseudo_dir': {
        'value': str(PSEUDO_DIR),
        'description': 'Directory containing pseudopotential files'
    },
    'verbosity': {
        'value': 'high',
        'description': 'Output detail level',
        'options': ['low', 'high']
    },
    'tprnfor': {
        'value': '.true.',
        'description': 'Print forces on atoms'
    },
    'tstress': {
        'value': '.true.',
        'description': 'Print stress tensor'
    },
    'etot_conv_thr': {
        'value': '1.0e-5',
        'description': 'Total energy convergence threshold for ionic relaxation (Ry)'
    },
    'forc_conv_thr': {
        'value': '1.0e-4',
        'description': 'Force convergence threshold for ionic relaxation (Ry/Bohr)'
    }
}

print("&CONTROL Namelist Parameters")
print("=" * 70)
for param, info in control_params.items():
    print(f"\n{param} = {info['value']}")
    print(f"  → {info['description']}")
    if 'options' in info:
        print(f"  Options: {info['options']}")

### 2.2 The &SYSTEM Namelist

This namelist defines the **physical system**: crystal structure, electrons, and basis set.

In [ ]:
system_params = {
    'ibrav': {
        'value': 2,
        'description': 'Bravais lattice index (2 = FCC)',
        'note': 'ibrav=0 allows custom cell via CELL_PARAMETERS card'
    },
    'celldm(1)': {
        'value': 10.26,
        'description': 'Lattice parameter a in Bohr (5.43 Å for Si)',
        'note': '1 Bohr = 0.529177 Å'
    },
    'nat': {
        'value': 2,
        'description': 'Number of atoms in the unit cell'
    },
    'ntyp': {
        'value': 1,
        'description': 'Number of different atomic species'
    },
    'ecutwfc': {
        'value': 40.0,
        'description': 'Kinetic energy cutoff for wavefunctions (Ry)',
        'note': 'CRITICAL: Must be converged! Start with PP suggestion.'
    },
    'ecutrho': {
        'value': 320.0,
        'description': 'Kinetic energy cutoff for charge density (Ry)',
        'note': 'Default = 4×ecutwfc (NC), 8-12×ecutwfc (US/PAW)'
    },
    'occupations': {
        'value': 'smearing',
        'description': 'How electronic states are occupied',
        'options': ['smearing', 'fixed', 'tetrahedra']
    },
    'smearing': {
        'value': 'cold',
        'description': 'Type of smearing function',
        'options': ['gaussian', 'methfessel-paxton', 'marzari-vanderbilt (cold)', 'fermi-dirac']
    },
    'degauss': {
        'value': 0.01,
        'description': 'Smearing width in Ry',
        'note': '0.01-0.02 Ry typical for semiconductors/metals'
    }
}

print("&SYSTEM Namelist Parameters")
print("=" * 70)
for param, info in system_params.items():
    print(f"\n{param} = {info['value']}")
    print(f"  → {info['description']}")
    if 'note' in info:
        print(f"  Note: {info['note']}")
    if 'options' in info:
        print(f"  Options: {info['options']}")

### 2.3 The &ELECTRONS Namelist

This namelist controls the **SCF self-consistency cycle**.

In [ ]:
electrons_params = {
    'conv_thr': {
        'value': '1.0e-8',
        'description': 'SCF convergence threshold on total energy (Ry)',
        'note': '1e-6 for quick tests, 1e-8 to 1e-10 for production'
    },
    'mixing_beta': {
        'value': 0.7,
        'description': 'Mixing factor for charge density update',
        'note': '0.3-0.7 typical; lower values for problematic convergence'
    },
    'mixing_mode': {
        'value': 'plain',
        'description': 'Charge mixing algorithm',
        'options': ['plain', 'TF', 'local-TF']
    },
    'diagonalization': {
        'value': 'david',
        'description': 'Algorithm for eigenvalue problem',
        'options': ['david', 'cg', 'ppcg']
    },
    'electron_maxstep': {
        'value': 100,
        'description': 'Maximum number of SCF iterations',
        'note': 'Increase if convergence is slow'
    }
}

print("&ELECTRONS Namelist Parameters")
print("=" * 70)
for param, info in electrons_params.items():
    print(f"\n{param} = {info['value']}")
    print(f"  → {info['description']}")
    if 'note' in info:
        print(f"  Note: {info['note']}")
    if 'options' in info:
        print(f"  Options: {info['options']}")

### 2.4 The Cards: ATOMIC_SPECIES, ATOMIC_POSITIONS, K_POINTS

In [ ]:
print("ATOMIC_SPECIES Card")
print("=" * 70)
print("""
Format: symbol  atomic_mass  pseudopotential_filename

Example:
    Si  28.0855  Si.upf

Notes:
- The symbol must match what's used in ATOMIC_POSITIONS
- Atomic mass in atomic mass units (amu)
- Pseudopotential file must exist in pseudo_dir
""")

print("\nATOMIC_POSITIONS Card")
print("=" * 70)
print("""
Format: ATOMIC_POSITIONS {unit}
        symbol  x  y  z

Units:
- {crystal}  : Fractional coordinates (a1, a2, a3)
- {angstrom} : Cartesian in Angstrom
- {bohr}     : Cartesian in Bohr
- {alat}     : Cartesian in units of celldm(1)

Example (diamond Si in crystal coords):
ATOMIC_POSITIONS {crystal}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25
""")

print("\nK_POINTS Card")
print("=" * 70)
print("""
Format: K_POINTS {type}

Types:
- {automatic}   : Monkhorst-Pack grid
    nx ny nz  sx sy sz
    (grid dimensions and shifts, 0 or 1)

- {gamma}       : Gamma point only (very fast, for large cells)

- {crystal}     : Explicit list in crystal coordinates
    nk
    k1x k1y k1z w1
    k2x k2y k2z w2
    ...

- {crystal_b}   : Band structure path (for 'bands' calculation)
    npath
    k1x k1y k1z npts
    k2x k2y k2z npts
    ...

Example (4×4×4 grid):
K_POINTS {automatic}
    4 4 4 0 0 0
""")

---

## 3. Creating the Complete Input File

In [ ]:
def generate_scf_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir, 
                       celldm1=10.26, conv_thr=1.0e-8):
    """
    Generate a complete SCF input file for bulk Silicon.
    
    Parameters
    ----------
    prefix : str
        Prefix for output files
    ecutwfc : float
        Wavefunction cutoff in Ry
    ecutrho : float
        Charge density cutoff in Ry
    kpoints : tuple
        (kx, ky, kz) Monkhorst-Pack grid dimensions
    pseudo_dir : str or Path
        Path to pseudopotential directory
    celldm1 : float
        Lattice parameter in Bohr (default: 10.26 for Si)
    conv_thr : float
        SCF convergence threshold in Ry
    
    Returns
    -------
    str
        Complete input file content
    """
    kx, ky, kz = kpoints
    
    input_text = f"""&CONTROL
    calculation = 'scf'
    prefix = '{prefix}'
    outdir = './tmp'
    pseudo_dir = '{pseudo_dir}'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/

&SYSTEM
    ibrav = 2
    celldm(1) = {celldm1}
    nat = 2
    ntyp = 1
    ecutwfc = {ecutwfc}
    ecutrho = {ecutrho}
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = {conv_thr}
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Si  28.0855  Si.upf

ATOMIC_POSITIONS {{crystal}}
    Si  0.00  0.00  0.00
    Si  0.25  0.25  0.25

K_POINTS {{automatic}}
    {kx} {ky} {kz} 0 0 0
"""
    return input_text

# Generate and display the input file
scf_input = generate_scf_input(
    prefix='silicon',
    ecutwfc=40.0,
    ecutrho=320.0,
    kpoints=(6, 6, 6),
    pseudo_dir=PSEUDO_DIR,
    conv_thr=1.0e-8
)

print("Generated SCF Input File:")
print("=" * 60)
print(scf_input)

In [ ]:
# Write the input file
input_file = WORK_DIR / 'silicon_scf.in'
with open(input_file, 'w') as f:
    f.write(scf_input)

print(f"Input file written to: {input_file}")

---

## 4. Running the SCF Calculation

In [ ]:
import time

def run_pwscf(input_file, output_file=None, nproc=1):
    """
    Run pw.x calculation.
    
    Parameters
    ----------
    input_file : Path
        Input file path
    output_file : Path, optional
        Output file path (default: same as input with .out extension)
    nproc : int
        Number of MPI processes (default: 1)
    
    Returns
    -------
    tuple
        (output_text, elapsed_time, success)
    """
    input_file = Path(input_file)
    if output_file is None:
        output_file = input_file.with_suffix('.out')
    
    work_dir = input_file.parent
    
    print(f"Running: pw.x < {input_file.name}")
    print(f"Working directory: {work_dir}")
    
    start_time = time.time()
    
    if nproc > 1:
        cmd = ['mpirun', '-np', str(nproc), 'pw.x', '-in', input_file.name]
    else:
        cmd = ['pw.x', '-in', input_file.name]
    
    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        cwd=work_dir,
        timeout=600  # 10 minute timeout
    )
    
    elapsed = time.time() - start_time
    
    # Write output to file
    with open(output_file, 'w') as f:
        f.write(result.stdout)
    
    success = 'convergence has been achieved' in result.stdout
    
    print(f"Elapsed time: {elapsed:.2f} s")
    print(f"Output saved to: {output_file}")
    
    if success:
        print("✓ SCF converged successfully!")
    else:
        print("✗ SCF did not converge or error occurred")
        if result.stderr:
            print(f"Errors: {result.stderr[:500]}")
    
    return result.stdout, elapsed, success

# Run the calculation
output_text, elapsed, success = run_pwscf(input_file)

---

## 5. Parsing the Output File

The output file contains a wealth of information. Let's parse it systematically.

In [ ]:
class QEOutputParser:
    """
    Parser for Quantum ESPRESSO pw.x output files.
    """
    
    def __init__(self, output_text):
        self.text = output_text
        self.lines = output_text.split('\n')
        self.results = {}
        self._parse()
    
    def _parse(self):
        """Parse all quantities from the output."""
        self._parse_system_info()
        self._parse_scf_iterations()
        self._parse_final_energy()
        self._parse_forces()
        self._parse_stress()
        self._parse_timing()
    
    def _parse_system_info(self):
        """Parse system information."""
        for line in self.lines:
            if 'lattice parameter (alat)' in line:
                match = re.search(r'=\s+([\d.]+)', line)
                if match:
                    self.results['alat_bohr'] = float(match.group(1))
                    self.results['alat_angstrom'] = float(match.group(1)) * 0.529177
            
            if 'unit-cell volume' in line:
                match = re.search(r'=\s+([\d.]+)', line)
                if match:
                    self.results['volume_bohr3'] = float(match.group(1))
                    self.results['volume_angstrom3'] = float(match.group(1)) * 0.529177**3
            
            if 'number of atoms/cell' in line:
                match = re.search(r'=\s+(\d+)', line)
                if match:
                    self.results['nat'] = int(match.group(1))
            
            if 'number of electrons' in line:
                match = re.search(r'=\s+([\d.]+)', line)
                if match:
                    self.results['nelec'] = float(match.group(1))
            
            if 'kinetic-energy cutoff' in line:
                match = re.search(r'=\s+([\d.]+)', line)
                if match:
                    self.results['ecutwfc'] = float(match.group(1))
            
            if 'charge density cutoff' in line:
                match = re.search(r'=\s+([\d.]+)', line)
                if match:
                    self.results['ecutrho'] = float(match.group(1))
            
            if 'number of k points' in line:
                match = re.search(r'=\s+(\d+)', line)
                if match:
                    self.results['nkpts'] = int(match.group(1))
    
    def _parse_scf_iterations(self):
        """Parse SCF iteration data."""
        iterations = []
        for line in self.lines:
            if 'total energy' in line and 'estimated' not in line and '!' not in line:
                match = re.search(r'total energy\s+=\s+([\d.E+-]+)', line)
                if match:
                    energy = float(match.group(1))
                    iterations.append(energy)
        
        self.results['scf_energies'] = iterations
        self.results['n_iterations'] = len(iterations)
    
    def _parse_final_energy(self):
        """Parse final total energy."""
        for line in self.lines:
            if '!' in line and 'total energy' in line:
                match = re.search(r'=\s+([\d.E+-]+)\s+Ry', line)
                if match:
                    self.results['total_energy_ry'] = float(match.group(1))
                    self.results['total_energy_ev'] = float(match.group(1)) * 13.605693
    
    def _parse_forces(self):
        """Parse atomic forces."""
        forces = []
        in_forces_section = False
        
        for line in self.lines:
            if 'Forces acting on atoms' in line:
                in_forces_section = True
                continue
            
            if in_forces_section:
                if 'atom' in line and 'force' in line:
                    # Parse: atom    1 type  1   force =     0.00000000    0.00000000    0.00000000
                    match = re.search(r'force\s+=\s+([\d.E+-]+)\s+([\d.E+-]+)\s+([\d.E+-]+)', line)
                    if match:
                        fx = float(match.group(1))
                        fy = float(match.group(2))
                        fz = float(match.group(3))
                        forces.append([fx, fy, fz])
                
                if 'Total force' in line:
                    match = re.search(r'Total force\s+=\s+([\d.E+-]+)', line)
                    if match:
                        self.results['total_force'] = float(match.group(1))
                    in_forces_section = False
        
        if forces:
            self.results['forces'] = np.array(forces)
    
    def _parse_stress(self):
        """Parse stress tensor."""
        stress = []
        in_stress_section = False
        
        for line in self.lines:
            if 'total   stress' in line:
                in_stress_section = True
                # Extract pressure from this line
                match = re.search(r'P=\s*([\d.E+-]+)', line)
                if match:
                    self.results['pressure_kbar'] = float(match.group(1))
                continue
            
            if in_stress_section:
                parts = line.split()
                if len(parts) == 6:
                    # First 3 are in Ry/bohr^3, last 3 are in kbar
                    stress.append([float(parts[3]), float(parts[4]), float(parts[5])])
                if len(stress) == 3:
                    in_stress_section = False
        
        if stress:
            self.results['stress_kbar'] = np.array(stress)
    
    def _parse_timing(self):
        """Parse timing information."""
        for line in self.lines:
            if 'PWSCF' in line and 'CPU' in line:
                # Parse: PWSCF        :     10.52s CPU     11.23s WALL
                match = re.search(r'([\d.]+)s CPU\s+([\d.]+)s WALL', line)
                if match:
                    self.results['cpu_time'] = float(match.group(1))
                    self.results['wall_time'] = float(match.group(2))
    
    def summary(self):
        """Print a formatted summary."""
        print("="*60)
        print("QUANTUM ESPRESSO SCF CALCULATION SUMMARY")
        print("="*60)
        
        print("\n--- System Parameters ---")
        if 'alat_bohr' in self.results:
            print(f"Lattice parameter: {self.results['alat_bohr']:.4f} Bohr = {self.results['alat_angstrom']:.4f} Å")
        if 'volume_angstrom3' in self.results:
            print(f"Unit cell volume: {self.results['volume_angstrom3']:.4f} Å³")
        if 'nat' in self.results:
            print(f"Number of atoms: {self.results['nat']}")
        if 'nelec' in self.results:
            print(f"Number of electrons: {self.results['nelec']:.1f}")
        if 'ecutwfc' in self.results:
            print(f"Wavefunction cutoff: {self.results['ecutwfc']:.1f} Ry")
        if 'ecutrho' in self.results:
            print(f"Charge density cutoff: {self.results['ecutrho']:.1f} Ry")
        if 'nkpts' in self.results:
            print(f"Number of k-points: {self.results['nkpts']}")
        
        print("\n--- SCF Convergence ---")
        if 'n_iterations' in self.results:
            print(f"SCF iterations: {self.results['n_iterations']}")
        
        print("\n--- Final Results ---")
        if 'total_energy_ry' in self.results:
            print(f"Total energy: {self.results['total_energy_ry']:.8f} Ry")
            print(f"             {self.results['total_energy_ev']:.6f} eV")
            if 'nat' in self.results:
                e_per_atom = self.results['total_energy_ev'] / self.results['nat']
                print(f"Energy/atom: {e_per_atom:.6f} eV")
        
        if 'total_force' in self.results:
            print(f"Total force: {self.results['total_force']:.6f} Ry/Bohr")
        
        if 'pressure_kbar' in self.results:
            print(f"Pressure: {self.results['pressure_kbar']:.2f} kbar")
            print(f"         {self.results['pressure_kbar'] * 0.1:.2f} GPa")
        
        print("\n--- Timing ---")
        if 'wall_time' in self.results:
            print(f"Wall time: {self.results['wall_time']:.2f} s")
        if 'cpu_time' in self.results:
            print(f"CPU time: {self.results['cpu_time']:.2f} s")
        
        print("="*60)

In [ ]:
# Parse the output
parser = QEOutputParser(output_text)
parser.summary()

---

## 6. Visualizing SCF Convergence

In [ ]:
# Plot SCF convergence
if 'scf_energies' in parser.results and len(parser.results['scf_energies']) > 1:
    energies = np.array(parser.results['scf_energies'])
    iterations = np.arange(1, len(energies) + 1)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot 1: Absolute energy vs iteration
    ax1 = axes[0]
    ax1.plot(iterations, energies, 'bo-', markersize=8, linewidth=2)
    ax1.set_xlabel('SCF Iteration', fontsize=12)
    ax1.set_ylabel('Total Energy (Ry)', fontsize=12)
    ax1.set_title('SCF Energy vs Iteration', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Energy difference (convergence)
    ax2 = axes[1]
    if len(energies) > 1:
        energy_diff = np.abs(np.diff(energies))
        ax2.semilogy(iterations[1:], energy_diff, 'ro-', markersize=8, linewidth=2)
        ax2.axhline(y=1e-8, color='g', linestyle='--', label='conv_thr = 1e-8 Ry')
        ax2.set_xlabel('SCF Iteration', fontsize=12)
        ax2.set_ylabel('|ΔE| (Ry)', fontsize=12)
        ax2.set_title('SCF Convergence', fontsize=14)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(str(WORK_DIR / 'scf_convergence.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nFigure saved to: {WORK_DIR / 'scf_convergence.png'}")
else:
    print("Not enough SCF iterations to plot convergence.")

---

## 7. Understanding Forces and Stress

### Forces
Forces on atoms tell us if the structure is at equilibrium:
- Zero forces → atoms at equilibrium positions
- Non-zero forces → need structural relaxation

### Stress Tensor
The stress tensor tells us about pressure on the cell:
- Diagonal elements: normal stresses
- Off-diagonal elements: shear stresses
- Trace/3 = hydrostatic pressure

**Negative pressure** → cell wants to contract → lattice parameter too large
**Positive pressure** → cell wants to expand → lattice parameter too small

In [ ]:
# Analyze forces and stress
if 'forces' in parser.results:
    forces = parser.results['forces']
    print("Forces on atoms (Ry/Bohr):")
    print("-" * 50)
    for i, f in enumerate(forces):
        magnitude = np.linalg.norm(f)
        print(f"Atom {i+1}: [{f[0]:12.8f}, {f[1]:12.8f}, {f[2]:12.8f}]  |F| = {magnitude:.2e}")
    print(f"\nTotal force: {parser.results.get('total_force', 'N/A')}")
    print("\nInterpretation: Forces ≈ 0 → Structure is at equilibrium")

if 'stress_kbar' in parser.results:
    stress = parser.results['stress_kbar']
    print("\n" + "="*50)
    print("Stress tensor (kbar):")
    print("-" * 50)
    for row in stress:
        print(f"[{row[0]:10.4f} {row[1]:10.4f} {row[2]:10.4f}]")
    
    pressure = parser.results['pressure_kbar']
    print(f"\nPressure: {pressure:.2f} kbar = {pressure * 0.1:.4f} GPa")
    
    if pressure > 5:
        print("\nInterpretation: Positive pressure → Cell wants to EXPAND")
        print("                → Lattice parameter is TOO SMALL")
    elif pressure < -5:
        print("\nInterpretation: Negative pressure → Cell wants to CONTRACT")
        print("                → Lattice parameter is TOO LARGE")
    else:
        print("\nInterpretation: Pressure ≈ 0 → Cell is near equilibrium")

---

## 8. Key Output File Sections Reference

Here's what to look for in a pw.x output file:

In [ ]:
output_sections = """
QUANTUM ESPRESSO OUTPUT FILE - KEY SECTIONS
============================================

1. HEADER
   - Version, compilation info
   - Parallelization details

2. INPUT ECHO
   - Bravais lattice type and parameters
   - Cell vectors (if ibrav=0)
   - Atomic positions

3. SYSTEM SETUP
   - "lattice parameter (alat)" → celldm(1) in Bohr
   - "unit-cell volume" → Cell volume
   - "number of atoms/cell" → nat
   - "number of electrons" → Total electrons
   - "kinetic-energy cutoff" → ecutwfc
   - "charge density cutoff" → ecutrho

4. K-POINTS
   - "number of k points" → Total k-points (with symmetry reduction)
   - List of k-points and weights

5. SCF ITERATIONS (for each iteration)
   - "iteration #" → Iteration number
   - "total energy" → Energy at this iteration
   - "estimated scf accuracy" → Convergence measure

6. CONVERGENCE MESSAGE
   - "convergence has been achieved in N iterations"
   - If not present → SCF failed!

7. FINAL RESULTS
   - "!    total energy" → Final total energy (Ry)
   - "     Harris-Foulkes estimate" → Alternative energy estimate
   - "     one-electron contribution"
   - "     hartree contribution"
   - "     xc contribution"
   - "     ewald contribution"

8. FORCES (if tprnfor=.true.)
   - "Forces acting on atoms"
   - "atom N type T force = ..." for each atom
   - "Total force"

9. STRESS (if tstress=.true.)
   - "total   stress  (Ry/bohr**3)     (kbar)     P= ..."
   - 3x3 stress tensor

10. TIMING
    - "PWSCF : Xs CPU Xs WALL"
    - Breakdown by subroutine
"""

print(output_sections)

---

## Summary

In this notebook, we have:

1. ✓ Understood every parameter in the QE input file
2. ✓ Written a proper SCF input file for bulk Silicon
3. ✓ Ran an SCF calculation and monitored convergence
4. ✓ Parsed the output file to extract all key quantities
5. ✓ Visualized SCF convergence
6. ✓ Interpreted forces and stress

### Critical Understanding

**Before using ANY calculated property, you MUST verify:**
1. SCF has converged ("convergence has been achieved")
2. Wavefunction cutoff (ecutwfc) is converged
3. K-point sampling is converged
4. For property calculations: lattice parameter is optimized

### Next Notebook
→ **03_Ecutwfc_Convergence.ipynb**: Systematic convergence testing of the wavefunction cutoff